In [1]:
import pandas as pd
import numpy as np

In [2]:
team_feats = pd.read_csv("../data/team_feats.csv")
games = pd.read_csv("../data/games.csv")

In [3]:
games=games[games['season']>=2017].copy()

In [4]:
team_feats = pd.DataFrame(np.where(team_feats==0,np.nan,team_feats),
                    columns=team_feats.columns,
                    index=team_feats.index)

In [5]:
team_feats

,season,team,cnt,completions,attempts,passing_yards,passing_tds,interceptions,sacks,sack_yards,...,receiving_first_downs,receiving_epa,receiving_2pt_conversions,racr,target_share,air_yards_share,wopr,special_teams_tds,fantasy_points,fantasy_points_ppr
0,2017,ARI,733,13.157143,15.641602,156.058571,1.676459,1.881746,2.184757,14.418485,...,1.584688,0.120258,1.0,1.004994,0.074038,0.065225,0.154486,1.2,5.381914,6.770719
1,2017,ATL,598,9.159926,13.913235,104.035784,1.394537,1.352031,1.896635,12.927632,...,1.885596,0.689923,1.0,0.639588,0.086667,0.077431,0.183482,1.0,5.648232,7.24072
2,2017,BAL,711,14.284936,10.922222,151.567788,1.406566,1.221434,1.982692,11.410897,...,1.667101,-0.03192,1.0,1.497929,0.086732,0.080565,0.185999,1.0,4.671847,6.30085
3,2017,BUF,809,9.510417,12.3,151.989583,1.253968,1.083333,2.666667,17.12,...,1.843229,0.588333,1.0,0.730224,0.096685,0.093041,0.209547,1.0,4.649553,6.441404
4,2017,CAR,569,14.050931,22.299197,172.347755,1.578571,1.572464,2.311688,14.905405,...,1.419304,0.727956,1.0,0.391524,0.072732,0.08082,0.165619,1.0,4.902468,6.222306
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
251,2024,SEA,421,21.820175,33.396199,231.331871,1.736779,1.396825,3.271242,23.747475,...,1.903724,0.530054,1.0,2.307878,0.091102,0.077682,0.18986,NaN,5.38883,7.317606
252,2024,SF,579,10.356609,15.695936,122.067672,1.339394,1.456277,2.509921,17.972583,...,1.7273,0.474776,1.0,1.75479,0.095819,0.09318,0.208697,1.0,6.442166,7.950973
253,2024,TB,455,11.735021,13.956751,126.506329,1.390625,1.340909,2.547761,17.326493,...,1.645248,0.40152,1.0,0.731652,0.085945,0.071629,0.17518,NaN,4.817556,6.360406
254,2024,TEN,528,7.841111,12.922381,90.184444,1.5,1.166667,2.313492,16.397817,...,1.880512,0.721951,1.0,0.499783,0.108696,0.106534,0.237617,1.0,4.536923,6.363807


In [8]:
home_feats = team_feats.copy()
home_feats.columns = ['home_'+i for i in home_feats.columns]
home_feats.rename(mapper={'home_season':'season'},axis=1,inplace=True)

away_feats = team_feats.copy()
away_feats.columns = ['away_'+i for i in away_feats.columns]
away_feats.rename(mapper={'away_season':'season'},axis=1,inplace=True)

In [9]:
home_feats

,season,home_team,home_cnt,home_completions,home_attempts,home_passing_yards,home_passing_tds,home_interceptions,home_sacks,home_sack_yards,...,home_receiving_first_downs,home_receiving_epa,home_receiving_2pt_conversions,home_racr,home_target_share,home_air_yards_share,home_wopr,home_special_teams_tds,home_fantasy_points,home_fantasy_points_ppr
0,2017,ARI,733,13.157143,15.641602,156.058571,1.676459,1.881746,2.184757,14.418485,...,1.584688,0.120258,1.0,1.004994,0.074038,0.065225,0.154486,1.2,5.381914,6.770719
1,2017,ATL,598,9.159926,13.913235,104.035784,1.394537,1.352031,1.896635,12.927632,...,1.885596,0.689923,1.0,0.639588,0.086667,0.077431,0.183482,1.0,5.648232,7.24072
2,2017,BAL,711,14.284936,10.922222,151.567788,1.406566,1.221434,1.982692,11.410897,...,1.667101,-0.03192,1.0,1.497929,0.086732,0.080565,0.185999,1.0,4.671847,6.30085
3,2017,BUF,809,9.510417,12.3,151.989583,1.253968,1.083333,2.666667,17.12,...,1.843229,0.588333,1.0,0.730224,0.096685,0.093041,0.209547,1.0,4.649553,6.441404
4,2017,CAR,569,14.050931,22.299197,172.347755,1.578571,1.572464,2.311688,14.905405,...,1.419304,0.727956,1.0,0.391524,0.072732,0.08082,0.165619,1.0,4.902468,6.222306
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
251,2024,SEA,421,21.820175,33.396199,231.331871,1.736779,1.396825,3.271242,23.747475,...,1.903724,0.530054,1.0,2.307878,0.091102,0.077682,0.18986,NaN,5.38883,7.317606
252,2024,SF,579,10.356609,15.695936,122.067672,1.339394,1.456277,2.509921,17.972583,...,1.7273,0.474776,1.0,1.75479,0.095819,0.09318,0.208697,1.0,6.442166,7.950973
253,2024,TB,455,11.735021,13.956751,126.506329,1.390625,1.340909,2.547761,17.326493,...,1.645248,0.40152,1.0,0.731652,0.085945,0.071629,0.17518,NaN,4.817556,6.360406
254,2024,TEN,528,7.841111,12.922381,90.184444,1.5,1.166667,2.313492,16.397817,...,1.880512,0.721951,1.0,0.499783,0.108696,0.106534,0.237617,1.0,4.536923,6.363807


In [10]:
away_feats

,season,away_team,away_cnt,away_completions,away_attempts,away_passing_yards,away_passing_tds,away_interceptions,away_sacks,away_sack_yards,...,away_receiving_first_downs,away_receiving_epa,away_receiving_2pt_conversions,away_racr,away_target_share,away_air_yards_share,away_wopr,away_special_teams_tds,away_fantasy_points,away_fantasy_points_ppr
0,2017,ARI,733,13.157143,15.641602,156.058571,1.676459,1.881746,2.184757,14.418485,...,1.584688,0.120258,1.0,1.004994,0.074038,0.065225,0.154486,1.2,5.381914,6.770719
1,2017,ATL,598,9.159926,13.913235,104.035784,1.394537,1.352031,1.896635,12.927632,...,1.885596,0.689923,1.0,0.639588,0.086667,0.077431,0.183482,1.0,5.648232,7.24072
2,2017,BAL,711,14.284936,10.922222,151.567788,1.406566,1.221434,1.982692,11.410897,...,1.667101,-0.03192,1.0,1.497929,0.086732,0.080565,0.185999,1.0,4.671847,6.30085
3,2017,BUF,809,9.510417,12.3,151.989583,1.253968,1.083333,2.666667,17.12,...,1.843229,0.588333,1.0,0.730224,0.096685,0.093041,0.209547,1.0,4.649553,6.441404
4,2017,CAR,569,14.050931,22.299197,172.347755,1.578571,1.572464,2.311688,14.905405,...,1.419304,0.727956,1.0,0.391524,0.072732,0.08082,0.165619,1.0,4.902468,6.222306
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
251,2024,SEA,421,21.820175,33.396199,231.331871,1.736779,1.396825,3.271242,23.747475,...,1.903724,0.530054,1.0,2.307878,0.091102,0.077682,0.18986,NaN,5.38883,7.317606
252,2024,SF,579,10.356609,15.695936,122.067672,1.339394,1.456277,2.509921,17.972583,...,1.7273,0.474776,1.0,1.75479,0.095819,0.09318,0.208697,1.0,6.442166,7.950973
253,2024,TB,455,11.735021,13.956751,126.506329,1.390625,1.340909,2.547761,17.326493,...,1.645248,0.40152,1.0,0.731652,0.085945,0.071629,0.17518,NaN,4.817556,6.360406
254,2024,TEN,528,7.841111,12.922381,90.184444,1.5,1.166667,2.313492,16.397817,...,1.880512,0.721951,1.0,0.499783,0.108696,0.106534,0.237617,1.0,4.536923,6.363807


In [13]:
merge = games.merge(home_feats,on=['season','home_team'],how='left').merge(away_feats,on=['season','away_team'],how='left')

In [14]:
merge

,game_id,season,game_type,week,gameday,weekday,gametime,away_team,away_score,home_team,...,away_receiving_first_downs,away_receiving_epa,away_receiving_2pt_conversions,away_racr,away_target_share,away_air_yards_share,away_wopr,away_special_teams_tds,away_fantasy_points,away_fantasy_points_ppr
0,2017_01_KC_NE,2017,REG,1,2017-09-07,Thursday,20:30,KC,42.0,NE,...,1.629669,-0.04831,1.0,1.209937,0.082076,0.056295,0.161012,1.0,5.407751,7.047958
1,2017_01_NYJ_BUF,2017,REG,1,2017-09-10,Sunday,13:00,NYJ,12.0,BUF,...,1.598947,0.104416,1.0,0.944843,0.100111,0.096714,0.217379,1.0,4.614442,6.441325
2,2017_01_ATL_CHI,2017,REG,1,2017-09-10,Sunday,13:00,ATL,23.0,CHI,...,1.885596,0.689923,1.0,0.639588,0.086667,0.077431,0.183482,1.0,5.648232,7.24072
3,2017_01_BAL_CIN,2017,REG,1,2017-09-10,Sunday,13:00,BAL,20.0,CIN,...,1.667101,-0.03192,1.0,1.497929,0.086732,0.080565,0.185999,1.0,4.671847,6.30085
4,2017_01_PIT_CLE,2017,REG,1,2017-09-10,Sunday,13:00,PIT,21.0,CLE,...,1.871225,0.016389,1.0,0.875705,0.094328,0.087228,0.198842,1.0,6.552251,8.093293
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2191,2024_18_MIA_NYJ,2024,REG,18,2025-01-05,Sunday,13:00,MIA,NaN,NYJ,...,1.723365,0.234242,1.0,1.59573,0.073783,0.067001,0.1571,1.0,5.174679,6.906019
2192,2024_18_NYG_PHI,2024,REG,18,2025-01-05,Sunday,13:00,NYG,NaN,PHI,...,1.746669,0.17522,1.0,0.635843,0.088804,0.088617,0.195229,1.0,4.877307,6.811559
2193,2024_18_CIN_PIT,2024,REG,18,2025-01-05,Sunday,13:00,CIN,NaN,PIT,...,1.888534,0.622128,1.0,1.160328,0.091486,0.101594,0.207819,1.0,6.183427,8.088967
2194,2024_18_NO_TB,2024,REG,18,2025-01-05,Sunday,13:00,NO,NaN,TB,...,1.647693,0.303851,1.0,1.512943,0.09306,0.079737,0.194578,1.0,5.912832,7.633896


In [15]:
merge['spread_line']

0        8.0
1        7.0
2       -6.5
3        2.5
4      -10.0
        ... 
2191     NaN
2192     NaN
2193     NaN
2194     NaN
2195     NaN
Name: spread_line, Length: 2196, dtype: float64

In [16]:
merge['result']

0      -15.0
1        9.0
2       -6.0
3      -20.0
4       -3.0
        ... 
2191     NaN
2192     NaN
2193     NaN
2194     NaN
2195     NaN
Name: result, Length: 2196, dtype: float64